# Notebook de prueba

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

In [2]:
file = '../data/ventas.csv'

In [4]:
df = pd.read_csv(file)

In [ ]:
df

In [ ]:
import calendar

def transformar_fecha(periodo):
    # Convertir el periodo a string
    periodo_str = str(periodo)
    # Obtener el año y el mes
    year = int(periodo_str[:4])
    month = int(periodo_str[4:6])
    # Obtener el último día del mes
    last_day = calendar.monthrange(year, month)[1]
    # Formatear la fecha como YYYY-MM-DD y convertir a datetime64
    return pd.to_datetime(f"{year}-{month:02d}-{last_day:02d}")

# Aplicar la función a la columna 'Fecha'
df['Fecha2'] = df['Fecha'].apply(transformar_fecha)
df

In [24]:
df['Producto_cliente'] = df['CodigoProducto'] + "_" + df['CodigoCliente']

In [25]:
tabla_df = df.pivot_table(index='Fecha2', columns='Producto_cliente', values='CantidadVendida', aggfunc='sum').reset_index()

In [ ]:
tabla_df.tail()

In [10]:
def generar_pronostico(df:pd.DataFrame, sales_column:str, periods_column:str, cant_periodos:int):
    # Preparar los datos para Prophet
    df_prophet = df[[periods_column, sales_column]].rename(columns={periods_column: 'ds', sales_column: 'y'})
    
    # Crear y ajustar el modelo
    modelo = Prophet()
    modelo.fit(df_prophet)
    
    # Crear un dataframe con las fechas futuras
    futuro = modelo.make_future_dataframe(periods=cant_periodos, freq='M')
    
    # Generar el pronóstico
    pronostico = modelo.predict(futuro)
    
    return pronostico

In [ ]:
# Generar un pronóstico para los próximos 12 meses
pronostico = generar_pronostico(df=tabla_df, sales_column = 'P001_C001', periods_column='Fecha2' , cant_periodos=36)


In [ ]:
pronostico.columns

In [ ]:
pronostico.plot(x='ds', y='yhat', label='Pronóstico', color='green', figsize=(15, 5))
plt.fill_between(pronostico['ds'], pronostico['yhat_lower'], pronostico['yhat_upper'], color='gray', alpha=0.3)
# pronostico.plot(x='ds', y='yhat_lower', label='Límite inferior', color='gray', ax=plt.gca())
# pronostico.plot(x='ds', y='yhat_upper', label='Límite superior', color='gray', ax=plt.gca())
tabla_df.plot(x='Fecha2', y='P001_C001', label='Histórico', color='black', ax=plt.gca())

In [ ]:
pronostico[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]